<a href="https://colab.research.google.com/github/OSGeoLabBp/tutorials/blob/master/hungarian/python/vectorization_hu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vektorizáció

A vektorizációt a PYthon programok felgyorsítására használjuk. Használata alapvető nagy adathlmazok kezelése esetén.

A vektorizált Python kódban nincsenek ciklusok (melyek lassan futnak), helyette összetett adatszerkezeteken végzett múveleteket használunk mint például *numpy* tömbök, *pandas* adatsorok. Ezeket a modulokat (*numpy*. *pandas*, stb.) C/C++ nyelven írták, melyek sokkal hatékonyabban hajtják végre a szükséges ciklusokat.

A vektorizált kód nem csak gyorsabban fut, de a kód rövidebb, könnyebben olvasható és karbantartható lesz.

Nézzünk meg néhány példát vektorizálás nélküli és vektorizált megoldással.

Nagy vektorokat és mátrixokat használunk, hogy a két megoldás közötti különbség látványosabb legyen. A műveletek eredményét általában nem írjuk ki, hanem csak a futási időket.



## Vektor és skalár szorzet


Tíz millió elemből álló vektort szorzunk meg egy skalárral.

In [1]:
import numpy as np
import random
import time

# adatok inicializálása
n = 10_000_000              # vektor mérete
scalar = 2.564              # Skalár szorzó a vektorhoz

In [2]:
%%time
vlist = [random.random() for i in range(n)]  # véletszámokat tartalmazó vektor generálása (nem vektorizált)

CPU times: user 1.34 s, sys: 227 ms, total: 1.56 s
Wall time: 1.61 s


In [3]:
%%time
vect = np.random.rand(n)     # véletszámokat tartalmazó vektor generálása (vektorizált)

CPU times: user 152 ms, sys: 23.9 ms, total: 176 ms
Wall time: 259 ms


Nem vektorizált megoldás egyszerű ciklussal.

In [4]:
%%time
slist = []
for i in range(n):
    slist.append(vlist[i] * scalar)

CPU times: user 3.07 s, sys: 358 ms, total: 3.43 s
Wall time: 4.1 s


Nem vektorizált megoldás lista feldolgozással.

In [5]:
%%time
s1list = [v * scalar for v in vlist]

CPU times: user 909 ms, sys: 359 ms, total: 1.27 s
Wall time: 1.31 s


Vektorizált megoldás numpy tömbbel.

In [6]:
%%time
svect = vect * scalar

CPU times: user 12.8 ms, sys: 26.5 ms, total: 39.3 ms
Wall time: 40.8 ms


## Legnagyobb elem megkeresése a vektorban

Nem vektorizált megoldás Python ciklussal.

In [7]:
%%time
vmax = vlist[0]
for v in vlist[1:]:
    if v > vmax: vmax = v

CPU times: user 1.6 s, sys: 67.9 ms, total: 1.67 s
Wall time: 2.49 s


Vektorizált megoldás listával.

In [8]:
%%time
vmax = max(vlist)

CPU times: user 154 ms, sys: 2.91 ms, total: 157 ms
Wall time: 158 ms


Vektorizált megoldás numpy tömbbel.

In [9]:
%%time
vmax = np.max(vect)

CPU times: user 10.9 ms, sys: 150 µs, total: 11 ms
Wall time: 14.2 ms


## Legnagyobb abszolút értékű különbség megkeresése egy vektorban

Nem vektorizált megoldás.

In [10]:
%%time
max_dif = abs(vlist[0] - vlist[1])
for i in range(1, n):
    dif = abs(vlist[i-1] - vlist[i])
    if dif > max_dif: max_dif = dif

CPU times: user 4.03 s, sys: 18.8 ms, total: 4.05 s
Wall time: 5.56 s


Vektorizált megoldás numpy tömbbel.

In [11]:
%%time
max_dif = np.max(np.abs(vect[:-1] - vect[1:]))

CPU times: user 31.3 ms, sys: 50.5 ms, total: 81.8 ms
Wall time: 84 ms


## Mátrix soronkénti átlagának számítása

Adatok előkészítése, a korábbi vektorból mátrix készítése.

In [12]:
matrix = vect.reshape((5000, -1)) # 5000 soros mátrix
list_matrix = list(matrix)        # átalakítás listává a nem vektorizált feldolgozáshoz

In [13]:
%%time
row_means = []
for row in list_matrix:
    row_means.append(sum(row)/ len(row))

CPU times: user 1.05 s, sys: 5.6 ms, total: 1.05 s
Wall time: 1.06 s


In [14]:
%%time
row_means = np.mean(matrix, axis=1)

CPU times: user 11.5 ms, sys: 0 ns, total: 11.5 ms
Wall time: 13.7 ms


## Területszámítás koordinátákból

Az alkalmazott képlet

$ 2 \cdot Area = \sum (x_i - x_{i+1}) \cdot (y_i + y_{i+1})$


Először létrehozunk egy sok töréspontból álló sokszöget.

In [15]:
from math import sin, cos, pi

Nem vektorizált megoldás az adatok generálására

In [16]:
%%time
# pontok generálása egy kör kerületén
c = []
r = 10
for i in range(0, 36000):   # Középponti szögek századfokokban
    x = r * sin(i / 18000 * pi)
    y = r * cos(i / 18000 * pi)
    c.append((x, y))

CPU times: user 304 ms, sys: 8.05 ms, total: 312 ms
Wall time: 313 ms


In [18]:
%%time
#nem vektorizált megoldás
s = 0
n = len(c)
for i in range(n):
    j = i+1 if i < n-1 else 0
    s += (c[i][0] - c[j][0]) * (c[i][1] + c[j][1])
area1 = abs(s / 2)
print(f'Terület: {area1:.5f}')

Terület: 314.15926
CPU times: user 25.1 ms, sys: 0 ns, total: 25.1 ms
Wall time: 26.4 ms


Vektorizált megoldás az adatok generálására.

In [20]:
%%time
angle = np.arange(0, 360, 0.01) / 180 * pi  # szög radiánkban
crds = np.hstack((np.sin(angle).reshape(-1, 1) * r, np.cos(angle).reshape(-1, 1) * r))
crds1 = np.vstack((crds, crds[0]))    # a végéhez illeszük oda az első sort

CPU times: user 3.91 ms, sys: 0 ns, total: 3.91 ms
Wall time: 4.35 ms


In [21]:
%%time
# vektorizált megoldás
area2 = abs(np.sum((crds1[:-1, 0] - crds1[1:, 0]) * (crds1[:-1, 1] + crds1[1:, 1]))) / 2
print(f'Terület {area2:.5f}')

Terület 314.15926
CPU times: user 1.65 ms, sys: 0 ns, total: 1.65 ms
Wall time: 5.36 ms


Hasonlítsa össze a vektorizált és nem vektorizált megoldások futási idejét!

##Feladatok

1. Számítsa ki a területszámítási példában használt pontok távolságát az origótól vektorizált megoldással!
2. Szintén vektorizált megoldással számítsa ki a szomszédos pontok távolságát!
3. Készítsen vektorizált megoldást egy vektor második differenciáinak előállítására!
4. Készítsen vektorizált megoldást egy négyzetes mátrix főátlójában lévő legnagyobb elem kikeresésére!
5. Készítsen vektorizált megoldást két vektor diadikus szorzatának előállítására!